### **Thông Tin Bài Nộp:**
- Nhóm: *04*
- Nhóm trưởng: *Phạm Nguyễn Hiền Phương*
- Môn học: *BIG DATA AND ITS APPLICATIONS*
- Giảng viên: *TS. Nguyễn Thôn Dã*
- Bài tập: *Bài Tập Nhóm Lần #1 - PySpark*

| StudentID      | Name | Class     |
| :---        |    :---:   |          :--- |
| K204110855 | Đinh Văn An | K20411 |
| K204110568 | Nguyễn Gia Hưng | K20411 |
| ***K204110579*** | ***Phạm Nguyễn Hiền Phương*** | ***K20411*** |
| K204111772 | Đinh Hoàn Hảo | K20411 |
| K204110567 | Phạm Thị Minh Hòa | K20411 |
| K194081062 | Lê Thanh Hải | K19408T |

In [ ]:
pip install PySpark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 51 kB/s 
     |████████████████████████████████| 199 kB 18.1 MB/s 
  Created wheel for PySpark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=f68d7fb0a4bf75dc136f15acc9b7c51e43b100e7e1f10e2a3e5cdefbd4a89f7e
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built PySpark


## **Import Libraries**

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import numpy as np
import pandas as pd

In [ ]:
spark = SparkSession.builder.appName('PySpark').getOrCreate()

## Mô tả Credit card fraud Dataset

| Field      | Description | Data Type |
| :---       |  :---       | :---      |
|transdatetrans_time|	Transaction DateTime | DateTime|
|merchant|	Merchant Name |    String|
|category|	Category of Merchant | String|
|amt|	Amount of Transaction | Float|
|city|	City of Credit Card Holder | String|
|state|	State of Credit Card Holder | String|
|lat|	Latitude Location of Purchase | Float|
|long|	Longitude Location of Purchase | Float|
|city_pop|	Credit Card Holder's City Population | Integer|
|job|	Job of Credit Card Holder | String|
|dob|	Date of Birth of Credit Card Holder | DateTime|
|trans_num|	Transaction Number | String|
|merch_lat|	Latitude Location of Merchant | Float|
|merch_long|	Longitude Location of Merchant | Float|
|is_fraud|	Whether Transaction is Fraud (1) or Not (0) | Integer|

## **4.1. TRANSFORM VALUES IN A COLUMN OF A DATAFRAME**

### **4.1.1. Tạo DataFrame**

Thực hiện load dữ liệu từ file `credit-card-fraud.csv` từ `Github` vào dataframe `is_frauddf` và kiểm tra kiểu dữ liệu của các cột trong dataframe.

In [ ]:
# file_location="Data\credit-card-fraud.csv"
# file_type="csv"
# infer_schema="False" 
# first_row_is_header="True"
# df_Origin = spark.read.format(file_type).option("inferSchema", infer_schema).option("header", first_row_is_header).load(file_location)

pandas_df = pd.read_csv('https://raw.githubusercontent.com/thanhENC/Data-set/main/credit-card-fraud.csv')
origin_df = spark.createDataFrame(pandas_df)
origin_df.show(5)
origin_df.printSchema()

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-1

**Casting**

Chuyển kiểu dữ liệu cho các cột về đúng kiểu dữ liệu đã được mô tả ở trên ***để tiến hành tính tuổi*** cho người dùng và tạo thành một cột dữ liệu mới.

In [ ]:
is_frauddf = origin_df

#identifying and assigning the correct data type to each column
float_vars = ['amt', 'lat', 'long', 'merch_lat', 'merch_long']
date_vars = ['dob']
datetime_vars = ['trans_date_trans_time']
int_vars = ['city_pop', 'is_fraud']
for var in float_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(DoubleType()))
for var in date_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(DateType()))
for var in datetime_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(TimestampType()))
for var in int_vars:
    is_frauddf = is_frauddf.withColumn(var, is_frauddf[var].cast(IntegerType()))

is_frauddf.printSchema()

root
 |-- trans_date_trans_time: timestamp (nullable = true)
 |-- merchant: string (nullable = true)
 |-- category: string (nullable = true)
 |-- amt: double (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- long: double (nullable = true)
 |-- city_pop: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- trans_num: string (nullable = true)
 |-- merch_lat: double (nullable = true)
 |-- merch_long: double (nullable = true)
 |-- is_fraud: integer (nullable = true)



In [ ]:
is_frauddf.count()

1333

Như vậy ta đã có dataframe `is_frauddf` với các cột đã được chuyển đổi kiểu dữ liệu và có tất cả 1333 dòng dữ liệu.

### **4.1.2. Tính Tuổi Của Card Holder và Thêm Thành Một Cột Dữ Liệu**

- Sử dụng lệnh cơ bản để tính tuổi (dateif) 

In [ ]:
is_frauddf = is_frauddf.withColumn('Age', datediff(lit(current_date()), is_frauddf['dob'])/lit(365))
is_frauddf.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+------------------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|               Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+------------------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44.33698630136986|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.26

### **4.1.3. Làm Tròn Giá Trị Của Cột Dữ Liệu Tuổi `Age`**

Làm tròn cột vừa mới tính toán( quay về tròn số)

In [45]:
is_frauddf = is_frauddf.withColumn('Age', floor(is_frauddf['Age']))
is_frauddf.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac

## **Scaling**

In [51]:
#Create tempView
is_frauddf.createOrReplaceTempView("transDf")

In [ ]:
#CACHE DATA
spark.sql("cache table transCached select * from transDf").show()

In [52]:
#Top 2 giao dịch ở các thành phố
spark.sql("select * from (select city, merchant, amt, rank() over (PARTITION BY city ORDER BY amt desc) as RANK from transDf) where RANK <= 2").show()

+-------------+--------------------+------+----+
|         city|            merchant|   amt|RANK|
+-------------+--------------------+------+----+
|  Albuquerque|   Champlin and Sons|132.62|   1|
|  Albuquerque|           Mohr-Baye|107.76|   2|
|      Altonah|        Ernser-Feest|143.15|   1|
|      Altonah|         Bradtke PLC|  73.2|   2|
|         Alva|        Treutel-King|456.68|   1|
|         Alva|       Stracke-Lemke| 99.48|   2|
|American Fork|    Denesik and Sons|512.29|   1|
|American Fork|  Haley Batz and Aue|138.28|   2|
|       Arnold|        Padberg-Saue|144.94|   1|
|       Arnold|Schaefer McGlynn ...| 90.29|   2|
|       Arvada|         Bins-Howell| 23.95|   1|
|       Arvada|Bradtke Torp and ...| 20.27|   2|
|      Ashford|Moen Reinger and ...| 93.01|   1|
|      Ashford|     Schuppe-Schuppe| 68.11|   2|
|       Athena|      Jast-McDermott|144.79|   1|
|       Athena|            Lesch Lt|132.09|   2|
|       Aurora|Eichmann Bogan an...| 87.77|   1|
|       Aurora|     

In [ ]:
#PARTITION BY
spark.sql("select city, amt, Age, rank() over (PARTITION BY city ORDER BY amt) as RANK from transDf").show()

+-----------+------+---+----+
|       city|   amt|Age|RANK|
+-----------+------+---+----+
|Albuquerque|  1.65| 46|   1|
|Albuquerque|   2.2| 55|   2|
|Albuquerque| 18.47| 46|   3|
|Albuquerque| 41.57| 55|   4|
|Albuquerque| 56.06| 46|   5|
|Albuquerque| 59.38| 46|   6|
|Albuquerque| 59.55| 46|   7|
|Albuquerque| 71.53| 55|   8|
|Albuquerque| 81.25| 55|   9|
|Albuquerque|107.76| 55|  10|
|Albuquerque|132.62| 55|  11|
|    Altonah|  2.74| 32|   1|
|    Altonah|  3.15| 32|   2|
|    Altonah|  4.24| 32|   3|
|    Altonah|  4.81| 32|   4|
|    Altonah|  7.67| 32|   5|
|    Altonah|   8.8| 32|   6|
|    Altonah| 13.02| 32|   7|
|    Altonah| 22.01| 32|   8|
|    Altonah| 34.74| 32|   9|
+-----------+------+---+----+
only showing top 20 rows



In [ ]:
spark.sql("select category, avg(amt) from transDf group by category").show()

+--------------+------------------+
|      category|          avg(amt)|
+--------------+------------------+
|        travel|175.87703703703698|
|      misc_net| 72.33287671232875|
|   grocery_pos|130.32606896551724|
|     kids_pets|56.600550458715595|
|  shopping_net|  92.3175238095238|
|   grocery_net|  57.6187037037037|
|   food_dining| 47.23774647887324|
| gas_transport| 64.27892857142858|
| personal_care| 51.59656565656565|
|health_fitness| 44.35176470588235|
| entertainment| 55.01124999999998|
|          home|63.603281249999995|
|      misc_pos|61.665180722891556|
|  shopping_pos| 35.45526315789473|
+--------------+------------------+



In [ ]:
spark.sql("select city, Age, count(Age) from transDf group by city, Age").show()

+-------------+---+----------+
|         city|Age|count(Age)|
+-------------+---+----------+
|    Louisiana| 81|         4|
|         Helm| 49|         9|
|   Mound City| 69|        11|
|Fort Washakie| 46|        14|
|   North Loup| 42|        14|
|     Roseland| 74|         5|
|   Broomfield| 55|         3|
|   Malad City| 60|         5|
|     San Jose| 67|         5|
| Kissee Mills| 42|         6|
|   Crownpoint| 33|         8|
|      Newberg| 45|         8|
| Indian Wells| 67|         9|
|    San Diego| 62|         3|
|      Howells| 94|         4|
|   Owensville| 24|         3|
|         Kent| 80|         5|
|    Fullerton| 49|         5|
|        Tekoa| 23|         8|
|   Fiddletown| 35|        17|
+-------------+---+----------+
only showing top 20 rows



## **4.2. CHỌN CỘT DỮ LIỆU TỪ DATAFRAME**

### Chọn cột lat và hiển thị ra 6 dòng tượng trưng

In [ ]:
df_Select = is_frauddf.select("long")
df_Select.show(6)

+---------+
|     long|
+---------+
|-118.2105|
| -112.262|
|-122.5258|
|-105.8189|
|-111.0292|
| -155.488|
+---------+
only showing top 6 rows



- Step 4-2-2. Selecting the id and lat
Columns: lấy 1 lúc nhiều cột ra

In [ ]:
df_Id = is_frauddf.select("long","merch_long")
df_Id.show(6)
#show 6 dòng tượng trưn, đại điệnđiện

+---------+-----------+
|     long| merch_long|
+---------+-----------+
|-118.2105|-118.186462|
| -112.262|-112.154481|
|-122.5258|-122.230347|
|-105.8189|-106.520205|
|-111.0292|-111.454923|
| -155.488|-156.045889|
+---------+-----------+
only showing top 6 rows



In [ ]:
df_mask = is_frauddf.select("long",is_frauddf.
long*2)
df_mask.show(6)

+---------+----------+
|     long|(long * 2)|
+---------+----------+
|-118.2105|  -236.421|
| -112.262|  -224.524|
|-122.5258| -245.0516|
|-105.8189| -211.6378|
|-111.0292| -222.0584|
| -155.488|  -310.976|
+---------+----------+
only showing top 6 rows



## **4.3. FILTER ROWA FROM A DATAFRAME PROBLE**

### **Step 4-3-1. Selecting Records Where the city Column Is Grenada**

In [ ]:
df_filter = is_frauddf.filter(is_frauddf.city == 'Grenada')
df_filter.show()

+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|   city|state|    lat|     long|city_pop|            job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:07:27|           Kiehn Inc|  grocery_pos| 96.29|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|413636e759663f264...| 41.65752|-122.230347|       0| 76|
|  2019-01-01 01:41:39|       Pacocha-Bauch| shopping_pos|   2.1|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|53fb041938a6e239a...|41.096733|-122.623454|       0| 76|
|  2019-01-01 03:45:30|Romaguera Cruicks...| shopp

### **4.3.2. Selecting Records Where category is "personal_care"**

In [ ]:
df_filter2 = is_frauddf.filter((is_frauddf.city ==
'Grenada') & (is_frauddf.category == 'personal_care'))
df_filter2.show()

+---------------------+--------------------+-------------+-----+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+----------+--------+---+
|trans_date_trans_time|            merchant|     category|  amt|   city|state|    lat|     long|city_pop|            job|       dob|           trans_num|merch_lat|merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+-----+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+----------+--------+---+
|  2019-01-01 12:41:17|Berge Kautzer and...|personal_care| 88.7|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|1f4c10dd875868477...|41.586814|-123.16098|       0| 76|
|  2019-01-01 19:54:19|Becker Harris and...|personal_care|29.24|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|f46cb7a27c537485a...|42.114385|-123.20183|       0| 76|
+---------------------+--------------------+-------------+--

### **Step 4-3-3. Selecting Records Where job Is Systems analys and merch_lat  > 41.3**

In [ ]:
is_frauddf.filter((is_frauddf.job ==
'Systems analys') & (is_frauddf.merch_lat > 41.5) )
df_filter2.show()

+---------------------+--------------------+-------------+-----+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+----------+--------+---+
|trans_date_trans_time|            merchant|     category|  amt|   city|state|    lat|     long|city_pop|            job|       dob|           trans_num|merch_lat|merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+-----+-------+-----+-------+---------+--------+---------------+----------+--------------------+---------+----------+--------+---+
|  2019-01-01 12:41:17|Berge Kautzer and...|personal_care| 88.7|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|1f4c10dd875868477...|41.586814|-123.16098|       0| 76|
|  2019-01-01 19:54:19|Becker Harris and...|personal_care|29.24|Grenada|   CA|41.6125|-122.5258|     589|Systems analyst|1945-12-21|f46cb7a27c537485a...|42.114385|-123.20183|       0| 76|
+---------------------+--------------------+-------------+--

## **4.4. DELETE A COLUMN FROM AN EXISTING DATAFRAME**

### 4.4.1. Dropping the is_Fraud from the df_Round DataFrame

In [ ]:
df_drop = is_frauddf.drop(is_frauddf.is_fraud)
df_drop.show(6)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-112.154481| 60|
|  20

### Step 4-4-2. Dropping the id and Occupation Columns from is_frauddf

In [ ]:
df_Drop2 = is_frauddf.drop("city", "city_pop")
df_Drop2.show(6)

+---------------------+--------------------+-------------+------+-----+-------+---------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|state|    lat|     long|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+-----+-------+---------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|   WA|48.8878|-118.2105|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|   ID|42.1808| -112.262|Nature conservati...|1962-01-19|a1a22d70485983eac...|43.150704|-112.154481|       0| 60|
|  2019-01-01 00:07:27|           Kiehn Inc|  grocery_pos| 96.29|   CA|41.6125|-122.5258|     Systems analyst|

## **4.5. TẠO VÀ SỬ DỤNG một PYSPARK SQL UDF**

In [ ]:
tempDf = is_frauddf
tempDf.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac

**Tạo UDF**

In [ ]:
def getBirthYear(x):
    return int(str(x).split('-')[0])

In [ ]:
getBirthYearUDF = udf(getBirthYear, IntegerType())

In [ ]:
tempDfBirthYear = tempDf.withColumn('Year', getBirthYearUDF(tempDf.dob))
tempDfBirthYear.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|Year|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|1978|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-

**Lưu kết quả vào file csv**

In [ ]:
#convert pyspark dataframe to pandas dataframe to save it to csv
tempDfBirthYear.toPandas().to_csv('tempBirthYear.csv', index=False)

## **4.6. GÁN NHÃN DỮ LIỆU**

In [ ]:
def labelBirthYear(year):
    if year >= 2010:
        return 'Gen Alpha'
    elif year >= 1997:
        return 'Gen Z'
    elif year >= 1980:
        return 'Millenials'
    elif year >= 1965:
        return 'Gen X'
    elif year >= 1945:
        return 'Baby Boomers'
    else:
        return 'Silent Generation'

In [ ]:
labelBirthYearUDF = udf(labelBirthYear, StringType())

In [ ]:
tempDfBirthYear2 = tempDfBirthYear.withColumn('Generation', labelBirthYearUDF(tempDfBirthYear.Year))
tempDfBirthYear2.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+------------+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|Year|  Generation|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+----+------------+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|1978|       Gen X|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1

## **4.7. Thống Kê Mô Tả**

**Sử dụng Pyspark để thực hiện một số mô tả với bộ dữ liệu hiện có**

*Đếm số merchant*

In [ ]:
countMer = is_frauddf.agg({'merchant':'count'})
countMer.show(5)

+---------------+
|count(merchant)|
+---------------+
|           1333|
+---------------+



*Tính số lược giao dịch trung bình*

In [ ]:
meanAmt = is_frauddf.agg({'amt':'avg'})
meanAmt.show()

+---------------+
|       avg(amt)|
+---------------+
|69.162273068267|
+---------------+



*Tính phương sai của cột giao dịch trung bình và tuổi của card holder*

In [ ]:
varAmt = is_frauddf.agg({'amt':'var_pop', 'Age':'var_pop'})
varAmt.show()

+------------------+-----------------+
|      var_pop(amt)|     var_pop(Age)|
+------------------+-----------------+
|15833.640752597596|278.3411726900216|
+------------------+-----------------+



*Để tiện lợi hơn, ta có thể tính các giá trị cần biết của cột lượt giao dịch trung bình trong một bảng. Ở đây ta tính các giá trị tổng, trung bình, phương sai và độ lệch chuẩn chuẩn*

In [ ]:
descriptiveAmt = is_frauddf.agg(sum("amt"), avg("amt"), var_pop('amt'), stddev_samp("amt"))
descriptiveAmt.show()

+-----------------+---------------+------------------+------------------+
|         sum(amt)|       avg(amt)|      var_pop(amt)|  stddev_samp(amt)|
+-----------------+---------------+------------------+------------------+
|92193.30999999991|69.162273068267|15833.640752597596|125.87902077105512|
+-----------------+---------------+------------------+------------------+



## **4.8. TÍNH COVARIANCE**

*Ta có thể tính covariance giữa cột lượt giao dịch trung bình và cột tuổi của card holder*

In [ ]:
is_frauddf.cov('amt','Age')

84.9300911094893

## **4.9. TÍNH CORRELATION**

*Ta có thể tính correlation giữa cột lượt giao dịch trung bình và cột tuổi của card holder*

In [ ]:
is_frauddf.corr('amt','Age')

0.040425606616528977

## **4.10. MÔ TẢ DATAFRAME**

Mô tả một DataFrame ứng dụng cho việc tính toán **thống kê đặc trưng** trên tất cả các cột trong một DataFrame. 

Trước tiên cần phải áp dụng hàm Des () lên từng cột của bảng

In [ ]:
dataDescription = is_frauddf.describe()
dataDescription.show()

+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+-------------------+--------------------+------------------+
|summary|            merchant|     category|               amt|                city|state|               lat|               long|         city_pop|                 job|           trans_num|        merch_lat|         merch_long|            is_fraud|               Age|
+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+-------------------+--------------------+------------------+
|  count|                1333|         1333|              1333|                1333| 1333|              1333|               1333|             1333|                1333|                1333|       

Hàm Describe( ) sẽ trả về kết quả của các thống kê đặc trưng. Ở cột 1, kết quả trả về là tên của các thống kê đặc trưng như là: giá trị lớn nhất, nhỏ nhất, độ lệch chuẩn, giá trị trung bình, đếm. Ở cột 4 tên là "amt" cung cấp giá trị cho các thống kê đặc trưng ví dụ như giá trị 1.05 là số giá trị nhỏ nhất trong cột amt. 

Có thể chọn cột bất kỳ, ví dụ như cột city và city_pop trong bảng DataFrame qua hàm sau: 

In [ ]:
dataDescriptioncitycitypop = is_frauddf.describe(['city', 'city_pop'])
dataDescriptioncitycitypop.show()

+-------+--------------------+-----------------+
|summary|                city|         city_pop|
+-------+--------------------+-----------------+
|  count|                1333|             1333|
|   mean|                null|96016.08552138034|
| stddev|                null|262532.5369151885|
|    min|         Albuquerque|               46|
|    max|Yellowstone Natio...|          1312922|
+-------+--------------------+-----------------+



Để có thể trả về những kết quả thống kê đặc trưng có chọn lọc hơn như là phương sai hoặc giá trị trung bình trên cột, ta không dùng hàm Describe( ) mà dùng hàm Summary( ) 

Hàm Summary( ) khá tương đồng với hàm Describe( ), tuy nhiên kết quả trả về có thêm điểm tứ phân vị (Quantiles) 25, 50, 75. 

In [ ]:
summaryData = is_frauddf.summary()
summaryData.show()

+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+-------------------+--------------------+------------------+
|summary|            merchant|     category|               amt|                city|state|               lat|               long|         city_pop|                 job|           trans_num|        merch_lat|         merch_long|            is_fraud|               Age|
+-------+--------------------+-------------+------------------+--------------------+-----+------------------+-------------------+-----------------+--------------------+--------------------+-----------------+-------------------+--------------------+------------------+
|  count|                1333|         1333|              1333|                1333| 1333|              1333|               1333|             1333|                1333|                1333|       

Cũng như hàm Describe( ), hàm Summary( ) cũng có thể chọn cột bất kỳ 

In [ ]:
summarystatejob = is_frauddf.select('state','job'). summary('min','max')
summarystatejob.show()

+-------+-----+--------------------+
|summary|state|                 job|
+-------+-----+--------------------+
|    min|   AK|Administrator edu...|
|    max|   WY|  Wellsite geologist|
+-------+-----+--------------------+



## **4.11. SẮP XẾP DỮ LIỆU TRONG DATAFRAME**

- Theo thứ tự tăng dần


   Sắp xếp bảng dữ liệu theo ngày giao dịch sinh tăng dần(sớm nhất) : 

In [ ]:
is_frauddfSorted1 = is_frauddf.orderBy("trans_date_trans_time")
is_frauddfSorted1.show(5)

+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|                city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+--------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 00:00:44|Heller Gutmann an...|  grocery_pos|107.23|              Orient|   WA|48.8878|-118.2105|     149|Special education...|1978-06-21|1f76529f857473494...|49.159047|-118.186462|       0| 44|
|  2019-01-01 00:00:51|      Lind-Buckridge|entertainment|220.11|          Malad City|   ID|42.1808| -112.262|    4154|Nature conservati...|1962-01-19|a1a22d70485983eac

- Theo thứ tự giảm dần

   Ngược lại với tăng dần ta có ngày giao dịch gần nhất: 

In [ ]:
is_frauddfSorted2 =is_frauddf.orderBy("trans_date_trans_time", ascending=False)
is_frauddfSorted2.show(5)

+---------------------+--------------+--------------+-----+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|      merchant|      category|  amt|       city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------+--------------+-----+-----------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-05 13:02:19|Dietrich-Fadel|health_fitness| 70.0|   Lakeport|   CA| 39.047|-122.9328|   11256|          Podiatrist|1972-10-18|726a1f0c5bc893784...|38.309802|-123.617622|       0| 50|
|  2019-01-05 12:42:03|    Barton LLC|     kids_pets|31.27|   Espanola|   NM|35.9866|-106.0654|   18408|Historic building...|1972-07-18|898f409ac52a44753...|35.178173|-106.125677|       0| 50|
|  2019-01-05 12:33:26|Erdman-Schad

- Sắp xếp 2 cột khác thứ tự

> Ví dụ như cột dân số tăng dần nhưng nghề nghiệp thì giảm dần(theo bảng chữ cái): 


In [ ]:
is_frauddfSorted3 = is_frauddf.orderBy("city_pop","job", ascending=[False,True])
is_frauddfSorted3.show(5)

+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+----------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|   amt|   city|state|    lat|     long|city_pop|       job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+------+-------+-----+-------+---------+--------+----------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-03 16:43:49|        Connelly PLC|  food_dining|130.72|Phoenix|   AZ|33.5623|-112.0559| 1312922|Contractor|1981-10-24|7a2ac4aa07d305ae3...| 34.08559|-111.774108|       0| 40|
|  2019-01-02 22:42:53|          Marvin-Lin|personal_care|  51.0|Phoenix|   AZ|33.5623|-112.0559| 1312922|Contractor|1981-10-24|4fc6ac61b3fe03c00...|33.067068|-112.630359|       0| 40|
|  2019-01-03 17:04:18|        Mayert Group| shopping_pos| 23.81|Phoenix|  

## **4.12 SẮP XẾP PHÂN VÙNG DỮ LIỆU**

-Chia dữ liệu thành 2 phân vùng: 

In [ ]:
is_frauddf = is_frauddf.repartition(2)
is_frauddf.rdd.glom().collect()

[[Row(trans_date_trans_time=datetime.datetime(2019, 1, 2, 19, 30, 32), merchant='Ledner Hartmann and Feest', category='health_fitness', amt=8.67, city='Manville', state='WY', lat=42.73, long=-104.7024, city_pop=241, job='Educational psychologist', dob=datetime.date(1980, 12, 16), trans_num='469ea411784baf6500dac4ff04e5371c', merch_lat=42.052084, merch_long=-104.828836, is_fraud=0, Age=41),
  Row(trans_date_trans_time=datetime.datetime(2019, 1, 2, 21, 22, 11), merchant='Lynch-Wisozk', category='home', amt=86.86, city='Lonetree', state='WY', lat=41.0552, long=-110.1603, city_pop=49, job='Research scientist (physical sciences)', dob=datetime.date(1981, 7, 5), trans_num='462d1556ef12345df4f68d27c48a8117', merch_lat=41.269291, merch_long=-110.942548, is_fraud=0, Age=41),
  Row(trans_date_trans_time=datetime.datetime(2019, 1, 1, 18, 19, 25), merchant='Crist Jakubowski and Littel', category='home', amt=107.64, city='Nelson', state='NE', lat=40.201, long=-98.0684, city_pop=732, job='Television

Sắp xếp ứng với 2 phân vùng đã phân

In [ ]:
sortedPartitons = is_frauddf.sortWithinPartitions("city_pop","job", ascending=[False,True])
sortedPartitons.show()

+---------------------+--------------------+--------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|   amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 19:03:05|         Baumbach Lt| personal_care|101.74|  Phoenix|   AZ|33.5623|-112.0559| 1312922|          Contractor|1981-10-24|faa5a1a33892b813d...|34.149154| -112.04887|       0| 40|
|  2019-01-01 18:46:36|       Stiedemann Lt|   food_dining|  66.7|  Phoenix|   AZ|33.5623|-112.0559| 1312922|          Contractor|1981-10-24|5e14511c0c8d96ee1...|32.674727|-112.497024|       0| 40|
|  2019-01

Sử dụng câu lệnh này để thể hiện rõ từng giá trị thuộc phần nào 

In [ ]:
sortedPartitons.rdd.glom().collect()

[[Row(trans_date_trans_time=datetime.datetime(2019, 1, 2, 19, 3, 5), merchant='Baumbach Lt', category='personal_care', amt=101.74, city='Phoenix', state='AZ', lat=33.5623, long=-112.0559, city_pop=1312922, job='Contractor', dob=datetime.date(1981, 10, 24), trans_num='faa5a1a33892b813d8d2c3ce13c992ca', merch_lat=34.149154, merch_long=-112.04887, is_fraud=0, Age=40),
  Row(trans_date_trans_time=datetime.datetime(2019, 1, 1, 18, 46, 36), merchant='Stiedemann Lt', category='food_dining', amt=66.7, city='Phoenix', state='AZ', lat=33.5623, long=-112.0559, city_pop=1312922, job='Contractor', dob=datetime.date(1981, 10, 24), trans_num='5e14511c0c8d96ee1ecd207815ed681d', merch_lat=32.674727, merch_long=-112.497024, is_fraud=0, Age=40),
  Row(trans_date_trans_time=datetime.datetime(2019, 1, 1, 9, 13, 31), merchant='Schmeler Inc', category='misc_pos', amt=2.72, city='Phoenix', state='AZ', lat=33.5623, long=-112.0559, city_pop=1312922, job='Contractor', dob=datetime.date(1981, 10, 24), trans_num='

## **4.13. XÓA CÁC DÒNG DỮ LIỆU TRÙNG LẶP**

**Việc xóa các dòng dữ liệu trùng lặp sẽ làm cho tệp dữ liệu được sạch hơn và giảm thiểu dung lượng lưu trữ.**

Để xóa các dòng trùng lặp, trước tiên ta tiến hành kiểm tra xem Dataset có các records bị trùng lặp hay không bằng lệnh count() và distinct.count()

In [ ]:
is_frauddf.count()

1333

In [ ]:
is_frauddf.distinct().count()

1333

Do cả 2 tệp dữ liệu đều có 1333 giá trị, như vậy, xem như tập dữ liệu không có các records bị trùng lặp. Tuy nhiên, ta có thể tự tạo ra các dòng trùng lặp như sau:

*Bước 1: Tạo ra một dòng dữ liệu random từ dataset*

In [ ]:
import random
frauddf_list = is_frauddf.collect()
generation = random.choice(frauddf_list)

*Bước 2: Tạo ra một dataframe mới có kế thừa từ dataframe cũ nhưng có thêm dòng dữ liệu vừa chọn. Như vậy xem như dataframe mới có 1 dòng dữ liệu bị trùng lặp*

In [ ]:
frauddf_list.append(generation)
header_list = is_frauddf.schema.names
dup_df = spark.createDataFrame(frauddf_list,header_list)
dup_df.show()

+---------------------+--------------------+--------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|   amt|          city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+------+--------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 19:30:32|Ledner Hartmann a...|health_fitness|  8.67|      Manville|   WY|  42.73|-104.7024|     241|Educational psych...|1980-12-16|469ea411784baf650...|42.052084|-104.828836|       0| 41|
|  2019-01-02 21:22:11|        Lynch-Wisozk|          home| 86.86|      Lonetree|   WY|41.0552|-110.1603|      49|Research scientis...|1981-07-05|462d1556ef12345df...|41.269291|-110.942548

*Bước 3: Kiểm tra Dataframe mới bằng hàm count() và distinct.count()*

In [ ]:
dup_df.count()

1334

In [ ]:
dup_df.distinct().count()

1333

Có thể thấy, Dataframe mới đã có 1 dòng dữ liệu bị duplicate. Sau đó tiến hành xóa dòng dữ liệu duplicate như sau:

In [ ]:
dup_df1 = dup_df.drop_duplicates()
dup_df1.show()

+---------------------+--------------------+--------------+------+-------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|   amt|         city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+------+-------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 21:22:23| Bartoletti and Sons| personal_care| 65.71| Laguna Hills|   CA| 33.615|-117.7114|   45303|     Sales executive|1969-12-22|c785a239482ff7c4a...|33.775845| -117.54871|       0| 52|
|  2019-01-02 16:18:08|Weber Thiel and H...|     kids_pets|  8.63|Weeping Water|   NE|40.8731| -96.1528|    1517|              Dealer|1951-11-08|150197506fc5fd84f...|41.295158| -96.117765|    

In [ ]:
dup_df1.count()

1333

**Giả sử muốn xóa các dữ liệu bị duplicate khi xét theo từng cột, ta làm như sau**

In [ ]:
dup_df2 = dup_df1.drop_duplicates(['category'])
dup_df2.show()

+---------------------+--------------------+--------------+-----+---------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|  amt|           city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+-----+---------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 17:48:36|         Schuppe LLC| entertainment|64.88|          Tekoa|   WA|47.2271|-117.0819|     895|Clothing/textile ...|1999-05-31|e3f75d3997d99a0e7...| 47.23421|-117.246013|       0| 23|
|  2019-01-01 07:13:01|       Jast and Sons|   food_dining|74.09|         Corona|   CA|33.8419|-117.6043|  233717|  Wellsite geologist|1966-05-22|4a4cf450e1ee2741e...|33.917816|-117.821826

In [ ]:
dup_df2.count()

14

Như vậy, do cột category chỉ có 14 dữ liệu riêng biệt, nên tệp dataframe sau khi xóa dữ liệu trùng lặp sẽ chỉ còn lại 14 dòng

**Tương tự, nếu muốn xóa các dòng dữ liệu riêng biệt khi xét trên tổ hợp 2 cột merchant và category, ta làm như sau:**

In [ ]:
dup_df3 = dup_df1.drop_duplicates(['merchant','category'])
dup_df3.show()

+---------------------+--------------------+--------------+-------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|    amt|              city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+-------+------------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 14:43:44|       Abbott-Rogahn| entertainment|  74.63|           Ravenna|   NE|41.0233| -98.9041|    2202|  Solicitor Scotland|1974-06-21|dbe29230128b03874...|41.874726| -99.842215|       0| 48|
|  2019-01-01 15:47:50|       Abbott-Steube| personal_care| 111.13|          Roseland|   NE|40.4591| -98.5551|     463| Nurse mental health|1948-05-31|40ca0f0111ef54d04

In [ ]:
dup_df3.count()

568

Như vậy, tệp dữ liệu chỉ còn 568 dòng tổ hợp khác nhau của hai cột category và merchant.

## **4.14. TRÍCH SAMPLE TỪ DATASET**

**Thao tác này cho phép tạo ra một tệp dữ liệu mới là sample (mẫu thu nhỏ) của tệp dataset ban đầu, với các dữ liệu được lấy random. Điều này thuận tiện cho người dùng khi muốn trích xuất các dữ liệu ngẫu nhiên để thống kê mô tả.**

**Giả sử muốn trích một sample có kích cỡ bằng 20% tệp dataset ban đầu, với điều kiện các dòng dữ liệu không trùng lặp, ta làm như sau:**

In [ ]:
fraud_sample = is_frauddf.sample(withReplacement=False,fraction=0.2,seed=200)
fraud_sample.show(5)

+---------------------+--------------------+--------------+------+-------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|   amt|         city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+------+-------------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 19:30:32|Ledner Hartmann a...|health_fitness|  8.67|     Manville|   WY|  42.73|-104.7024|     241|Educational psych...|1980-12-16|469ea411784baf650...|42.052084|-104.828836|       0| 41|
|  2019-01-01 13:07:33|Bahringer Bergnau...|          home|106.65|        Shedd|   OR| 44.453|-123.1065|     917|Radio broadcast a...|1995-07-08|c9df02be0b8e5207c...|43.617342|-122.715918|    

In [ ]:
fraud_sample.count()

291

Kết quả thu được 293 dữ liệu ngẫu nhiên, chiếm 20% tệp dữ liệu ban đầu.

**Giả sử muốn trích một sample có kích cỡ bằng 20% tệp dataset ban đầu, với điều kiện các dòng dữ liệu có thể trùng lặp, ta làm như sau:**

In [ ]:
fraud_sample1 = is_frauddf.sample(withReplacement=True,fraction=0.2,seed=200)
fraud_sample1.show(5)

+---------------------+--------------------+--------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|      category|   amt|     city|state|    lat|     long|city_pop|                 job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+--------------+------+---------+-----+-------+---------+--------+--------------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-02 19:30:32|Ledner Hartmann a...|health_fitness|  8.67| Manville|   WY|  42.73|-104.7024|     241|Educational psych...|1980-12-16|469ea411784baf650...|42.052084|-104.828836|       0| 41|
|  2019-01-01 18:19:25|Crist Jakubowski ...|          home|107.64|   Nelson|   NE| 40.201| -98.0684|     732|Television/film/v...|1956-12-13|0134e6a797bcf8c07...|39.879013| -97.392439|       0| 65|
|  2019-01

In [ ]:
fraud_sample1.count()

275

**Giả sử muốn trích một sample với hệ quy chiếu là trường category, với 30% tệp được lấy từ hình thức thanh toán ăn tối (food dining), và 70% tệp được lấy từ hình thức thanh toán xăng dầu (gas transport), ta làm như sau:**

In [ ]:
fraud_sample2 = is_frauddf.sampleBy(col='category',fractions={'food_dining':0.3,'gas_transport':0.7},seed=200)
fraud_sample2.show(5)

+---------------------+--------------------+-------------+-----+-------------+-----+-------+---------+--------+----------------+----------+--------------------+---------+-----------+--------+---+
|trans_date_trans_time|            merchant|     category|  amt|         city|state|    lat|     long|city_pop|             job|       dob|           trans_num|merch_lat| merch_long|is_fraud|Age|
+---------------------+--------------------+-------------+-----+-------------+-----+-------+---------+--------+----------------+----------+--------------------+---------+-----------+--------+---+
|  2019-01-01 02:08:13|Zieme Bode and Do...|gas_transport|52.67|  Kansas City|   MO|38.9621| -94.5959|  545147|      Counsellor|1987-11-18|29768857d3597be57...|39.662816| -93.712244|       0| 34|
|  2019-01-02 09:30:42|Connelly Reichert...|gas_transport|53.82|  Rocky Mount|   MO|38.2911| -92.7059|    1847|   Tax inspector|1988-10-26|e4a469a6382db84ee...|37.523602| -92.085787|       0| 33|
|  2019-01-02 10:33:

In [ ]:
fraud_sample2.count()

142

Như vậy, tệp sample có 145 dữ liệu, trong đó 30% tệp có hình thức thanh toán ăn tối (food dining), và 70% tệp có thanh toán xăng dầu (gas transport)

## **4.15. TÌM CÁC THÔNG TIN CÓ MẬT ĐỘ XUẤT HIỆN DÀY ĐẶC**

**Giả sử như muốn tìm xem các giao dịch được đến từ hình thức (category) nào nhiều nhất, ta làm như sau:**

In [ ]:
is_frauddf.freqItems(cols=['category']).show()

+--------------------+
|  category_freqItems|
+--------------------+
|[food_dining, per...|
+--------------------+



Như vậy, có thể thấy, các giao dịch đến nhiều nhất từ việc thanh toán ăn tối (food dining)

**Giả sử như muốn tìm xem các giao dịch được đến từ hình thức (category) nào nhiều nhất, và được thực hiện bởi ai nhiều nhất, ta làm như sau:**

In [ ]:
is_frauddf.freqItems(cols=['category','merchant']).show()

+--------------------+--------------------+
|  category_freqItems|  merchant_freqItems|
+--------------------+--------------------+
|[food_dining, per...|[Casper Hand and ...|
+--------------------+--------------------+



Như vậy, có thể thấy, các giao dịch đến nhiều nhất từ việc thanh toán ăn tối (food dining), và người thực hiện nhiều nhất là "Casper Hand and ...".